**IMPORTANTE: caso já tenha baixado a pasta colmap_project, inicie pela etapa 5.1 para as visualizações**

In [2]:
import pycolmap
from pathlib import Path
import shutil
import urllib.request
import zipfile
import io
import matplotlib.pyplot as plt
from PIL import Image

### Passo 1: Usando Imagens de uma Pasta Local

Esta etapa agora aponta para um diretório local chamado "imagens-t2/Imagens4".

Ação Necessária: Ter a pasta imagens-t2 com as imagens no mesmo diretório onde este notebook está salvo.

In [ ]:
# Define os caminhos para os conjuntos de imagens
image_paths = [
    Path('imagens-t2/Imagens1'),
    Path('imagens-t2/Imagens2'),
    Path('imagens-t2/Imagens3'),
    Path('imagens-t2/Imagens4')
]

for idx, image_path in enumerate(image_paths, start=1):
    project_path = Path(f'colmap_project_{idx}')
    database_path = project_path / 'database.db'
    output_path = project_path / 'sparse'

    # Limpa execuções anteriores para garantir um começo limpo
    if project_path.exists():
        shutil.rmtree(project_path)

    # Cria os diretórios para o projeto (exceto o de imagens, que será local)
    project_path.mkdir(exist_ok=True)
    output_path.mkdir(exist_ok=True)

    print(f"Diretórios do projeto para a base de dados e saídas criados em: {project_path.resolve()}")

    # Verifica se a pasta existe e não está vazia
    if not image_path.exists() or not any(image_path.iterdir()):
        print("="*80)
        print(f"ERRO: A pasta '{image_path}' não foi encontrada ou está vazia.")
        print("Por favor, crie esta pasta no mesmo diretório do notebook e adicione suas imagens.")
        print("="*80)
        # Lança um erro para parar a execução do notebook
        raise FileNotFoundError(f"A pasta de imagens '{image_path}' está faltando ou vazia.")
    else:
        print(f"Usando imagens da pasta local: '{image_path.resolve()}'")

    # Vamos visualizar algumas das imagens para confirmar
    image_files = sorted(list(image_path.glob('*.[jJ][pP][gG]')) + list(image_path.glob('*.[pP][nN][gG]')))
    print(f"Encontradas {len(image_files)} imagens.")

    fig, axes = plt.subplots(1, min(4, len(image_files)), figsize=(16, 4))
    if len(image_files) > 1:
        for j, img_file in enumerate(image_files[:4]):
            img = Image.open(img_file)
            axes[j].imshow(img)
            axes[j].set_title(img_file.name)
            axes[j].axis('off')
    elif len(image_files) == 1:
        img = Image.open(image_files[0])
        axes.imshow(img)
        axes.set_title(image_files[0].name)
        axes.axis('off')

    plt.show()

### Passo 2: Extração de Features (Keypoints)

Neste passo, o COLMAP analisará cada imagem para encontrar pontos de interesse distintos (features). Pelo que foi pesquisado, o algoritmo usado é o SIFT.

Atenção: Se você tiver uma GPU NVIDIA com CUDA configurado, pode definir use_gpu=True para uma aceleração massiva.

In [ ]:
# Configurações para a extração de features
feature_options = pycolmap.SiftExtractionOptions()
feature_options.use_gpu = True # Mude para True se tiver uma GPU compatível

# Itera sobre todos os conjuntos de imagens
def extract_features_for_all():
    for idx, image_path in enumerate(image_paths, start=1):
        project_path = Path(f'colmap_project_{idx}')
        database_path = project_path / 'database.db'

        print(f"Extraindo features para o conjunto {idx} localizado em: {image_path}")

        # Executa a extração
        pycolmap.extract_features(database_path, image_path, sift_options=feature_options)

        print(f"Extração de features concluída para o conjunto {idx}.")

extract_features_for_all()

W20250928 20:35:03.426201 140217776337984 feature_extraction.cc:404] Your current options use the maximum number of threads on the machine to extract features. Extracting SIFT features on the CPU can consume a lot of RAM per thread for large images. Consider reducing the maximum image size and/or the first octave or manually limit the number of extraction threads. Ignore this warning, if your machine has sufficient memory for the current settings.
I20250928 20:35:03.426797 140205123335744 misc.cc:44] 
Feature extraction
I20250928 20:35:03.428481 140204464059968 sift.cc:727] Creating SIFT CPU feature extractor
I20250928 20:35:03.428492 140204472452672 sift.cc:727] Creating SIFT CPU feature extractor
I20250928 20:35:03.428544 140204447274560 sift.cc:727] Creating SIFT CPU feature extractor
I20250928 20:35:03.428580 140204455667264 sift.cc:727] Creating SIFT CPU feature extractor
I20250928 20:35:03.428605 140204438881856 sift.cc:727] Creating SIFT CPU feature extractor
I20250928 20:35:03.

Extração de features concluída.


### Passo 3: Correspondência de Features (Matching)

Agora, o COLMAP irá comparar as features extraídas entre todas as imagens para encontrar correspondências (matches). Pelo que foi pesquisado, o algoritmo utilizado é: Nearest Neighbor + Verificação Geométrica com RANSAC.

In [ ]:
# Configurações para a correspondência
matcher_options = pycolmap.SiftMatchingOptions()
matcher_options.use_gpu = True # Mude para True se tiver uma GPU compatível

# Itera sobre todos os conjuntos de imagens
def match_features_for_all():
    for idx, image_path in enumerate(image_paths, start=1):
        project_path = Path(f'colmap_project_{idx}')
        database_path = project_path / 'database.db'

        print(f"Correspondendo features para o conjunto {idx} localizado em: {image_path}")

        # Executa a correspondência
        pycolmap.match_exhaustive(database_path, sift_options=matcher_options)

        print(f"Correspondência de features concluída para o conjunto {idx}.")

match_features_for_all()

I20250928 20:36:22.625818 140205123335744 misc.cc:44] 
Feature matching
I20250928 20:36:22.648662 140204065072704 sift.cc:1434] Creating SIFT CPU feature matcher
I20250928 20:36:22.649799 140204073465408 sift.cc:1434] Creating SIFT CPU feature matcher
I20250928 20:36:22.650700 140204081858112 sift.cc:1434] Creating SIFT CPU feature matcher
I20250928 20:36:22.651724 140205114943040 sift.cc:1434] Creating SIFT CPU feature matcher
I20250928 20:36:22.652733 140205106550336 sift.cc:1434] Creating SIFT CPU feature matcher
I20250928 20:36:22.653819 140205098157632 sift.cc:1434] Creating SIFT CPU feature matcher
I20250928 20:36:22.654770 140204929046080 sift.cc:1434] Creating SIFT CPU feature matcher
I20250928 20:36:22.655809 140204937438784 sift.cc:1434] Creating SIFT CPU feature matcher
I20250928 20:36:22.656830 140204920653376 sift.cc:1434] Creating SIFT CPU feature matcher
I20250928 20:36:22.657936 140204912260672 sift.cc:1434] Creating SIFT CPU feature matcher
I20250928 20:36:22.658883 14

Correspondência de features concluída.


I20250928 20:37:44.868425 140205123335744 feature_matching.cc:47] in 16.724s
I20250928 20:37:44.868515 140205123335744 timer.cc:91] Elapsed time: 1.371 [minutes]


### Passo 4: Reconstrução Esparsa (Structure from Motion)

Este é o passo principal. Usando as correspondências de features, o pycolmap executará o mapeamento incremental (SfM) para estimar simultaneamente:

- A estrutura 3D da cena (na forma de uma nuvem de pontos esparsa).

- Os parâmetros intrínsecos e extrínsecos da câmera para cada imagem (ou seja, onde cada foto foi tirada).

In [ ]:
# Itera sobre todos os conjuntos de imagens para realizar a reconstrução esparsa
def sparse_reconstruction_for_all():
    for idx, image_path in enumerate(image_paths, start=1):
        project_path = Path(f'colmap_project_{idx}')
        database_path = project_path / 'database.db'
        output_path = project_path / 'sparse'

        print(f"Iniciando reconstrução esparsa para o conjunto {idx} localizado em: {image_path}")

        # Executa o mapeamento incremental (reconstrução esparsa)
        reconstructions = pycolmap.incremental_mapping(database_path, image_path, output_path)

        print("Reconstrução esparsa concluída.")

        # Imprime um resumo do maior modelo reconstruído
        if reconstructions and 0 in reconstructions:
            print("\nResumo da Reconstrução:")
            print(reconstructions[0].summary())
        else:
            print("\nA reconstrução falhou. Verifique se as imagens têm sobreposição suficiente e boa qualidade.")

sparse_reconstruction_for_all()

I20250928 20:37:44.972383 140217776337984 incremental_pipeline.cc:254] Loading database
I20250928 20:37:45.015400 140217776337984 database_cache.cc:66] Loading rigs...
I20250928 20:37:45.016759 140217776337984 database_cache.cc:76]  1 in 0.001s
I20250928 20:37:45.016794 140217776337984 database_cache.cc:84] Loading cameras...
I20250928 20:37:45.017878 140217776337984 database_cache.cc:102]  1 in 0.001s
I20250928 20:37:45.017903 140217776337984 database_cache.cc:110] Loading frames...
I20250928 20:37:45.019257 140217776337984 database_cache.cc:127]  92 in 0.001s
I20250928 20:37:45.019281 140217776337984 database_cache.cc:135] Loading matches...
I20250928 20:37:45.621760 140217776337984 database_cache.cc:140]  4037 in 0.603s
I20250928 20:37:45.621813 140217776337984 database_cache.cc:156] Loading images...
I20250928 20:37:46.247609 140217776337984 database_cache.cc:241]  92 in 0.626s (connected 92)
I20250928 20:37:46.247667 140217776337984 database_cache.cc:252] Building correspondence g

Reconstrução esparsa concluída.

Resumo da Reconstrução:
Reconstruction:
	num_rigs = 1
	num_cameras = 1
	num_frames = 92
	num_reg_frames = 92
	num_images = 92
	num_points3D = 34347
	num_observations = 209810
	mean_track_length = 6.10854
	mean_observations_per_image = 2280.54
	mean_reprojection_error = 0.862968
